# Using crack submodels in PyBaMM
In this notebook we show how to use the crack submodel with battery DFN or SPM models. To see all of the models and submodels available in PyBaMM, please take a look at the documentation [here](https://docs.pybamm.org/en/latest/source/api/models/index.html).

In [ ]:
%pip install "pybamm[plot,cite]" -q    # install PyBaMM if it is not installed
import pybamm
import os
import matplotlib.pyplot as plt

os.chdir(pybamm.__path__[0] + "/..")

Then we load the DFN, SPMe or SPM, by choosing one and commenting the others. 

When you load a model in PyBaMM it builds by default. Building the model sets all of the model variables and sets up any variables which are coupled between different submodels: this is the process which couples the submodels together and allows one submodel to access variables from another. 

In [ ]:
model = pybamm.lithium_ion.DFN(
    options={
        "particle": "Fickian diffusion",
        "particle mechanics": "swelling and cracking",  # other options are "none", "swelling only"
    }
)

Load the parameter set Ai2020 which contains mechanical parameters. Other sets may not contain mechanical parameters should add them manually. 

In [ ]:
param = pybamm.ParameterValues("Ai2020")
## It can update the speed of crack propagation using the commands below:
# param.update({"Negative electrode Cracking rate":3.9e-20*10}, check_already_exists=False)

We can get the default parameters for the model and update them with the parameters required by the cracking model. Eventually, we would like these to be added to their own chemistry (you might need to adjust the path to the parameters file to your system).
Now the model can be processed and solved in the usual way, and we still have access to model defaults such as the default geometry and default spatial methods

Depending on the parameter set being used, the particle cracking model can require a large number of mesh points inside the particles to be numerically stable.

In [ ]:
var_pts = {
    "x_n": 20,  # negative electrode
    "x_s": 20,  # separator
    "x_p": 20,  # positive electrode
    "r_n": 26,  # negative particle
    "r_p": 26,  # positive particle
}

In [ ]:
sim = pybamm.Simulation(
    model,
    parameter_values=param,
    solver=pybamm.CasadiSolver(dt_max=600),
    var_pts=var_pts,
)
solution = sim.solve(t_eval=[0, 3600], inputs={"C-rate": 1})
# plot
quick_plot = pybamm.QuickPlot(solution)
quick_plot.dynamic_plot()

Plot the results as required.

In [ ]:
# extract voltage
E_n = param["Negative electrode Young's modulus [Pa]"]
stress_t_n_surf = solution["Negative particle surface tangential stress [Pa]"]
x = solution["x [m]"].entries[0:19, 0]
c_s_n = solution["Negative particle concentration"]
r_n = solution["r_n [m]"].entries[:, 0, 0]

# plot


def plot_concentrations(t):
    f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(20, 4))
    ax1.plot(x, stress_t_n_surf(t=t, x=x) / E_n)
    ax1.set_xlabel(r"$x_n$ [m]")
    ax1.set_ylabel("$\sigma_t/E_n$")

    (plot_c_n,) = ax2.plot(
        r_n, c_s_n(r=r_n, t=t, x=x[0])
    )  # can evaluate at arbitrary x (single representative particle)
    ax2.set_ylabel("Negative particle concentration")
    ax2.set_xlabel(r"$r_n$ [m]")
    ax2.set_ylim(0, 1)
    ax2.set_title("Close to current collector")
    ax2.grid()

    (plot_c_n,) = ax3.plot(
        r_n, c_s_n(r=r_n, t=t, x=x[10])
    )  # can evaluate at arbitrary x (single representative particle)
    ax3.set_ylabel("Negative particle concentration")
    ax3.set_xlabel(r"$r_n$ [m]")
    ax3.set_ylim(0, 1)
    ax3.set_title("In the middle")
    ax3.grid()

    (plot_c_n,) = ax4.plot(
        r_n, c_s_n(r=r_n, t=t, x=x[-1])
    )  # can evaluate at arbitrary x (single representative particle)
    ax4.set_ylabel("Negative particle concentration")
    ax4.set_xlabel(r"$r_n$ [m]")
    ax4.set_ylim(0, 1)
    ax4.set_title("Close to separator")
    ax4.grid()
    plt.show()


import ipywidgets as widgets

widgets.interact(
    plot_concentrations, t=widgets.FloatSlider(min=0, max=3600, step=10, value=0)
);

Plot results using the default functions

In [ ]:
label = ["Crack model"]
output_variables = [
    "Negative particle crack length [m]",
    "Positive particle crack length [m]",
    "X-averaged negative particle crack length [m]",
    "X-averaged positive particle crack length [m]",
]
quick_plot = pybamm.QuickPlot(
    solution, output_variables, label, variable_limits="tight"
)
quick_plot.dynamic_plot();

## References

The relevant papers for this notebook are:

In [ ]:
pybamm.print_citations()